# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [38]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.csv')

# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['MAT','LUK','JHN'])]
validate = bible[bible['book'].isin(['MRK'])]

In [39]:
train

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig
23213,MAT 1:1,MAT,1,1,"The book of the genealogy of Jesus Christ, the...","The book of the generation of Jesus Christ, th...","The book of the generation of Jesus Christ, th...","The book of the generations of Jesus Christ, t...","The book of the generacioun of Jhesu Crist, th...","Lxi fuuc us lxi girivekouar us Yxiza Kvozl, lx..."
23214,MAT 1:2,MAT,1,2,Abraham became the father of Isaac. Isaac beca...,Abraham begat Isaac; and Isaac begat Jacob; an...,Abraham begat Isaac; and Isaac begat Jacob; an...,The son of Abraham was Isaac; and the son of I...,Abraham bigat Isaac. Isaac bigat Jacob. Jacob ...,Efvexen fogel Ozeek. Ozeek fogel Yekuf. Yekuf ...
23215,MAT 1:3,MAT,1,3,Judah became the father of Perez and Zerah by ...,and Judah begat Perez and Zerah of Tamar; and ...,And Judas begat Phares and Zara of Thamar; and...,And the sons of Judah were Perez and Zerah by ...,"Judas bigat Fares and Zaram, of Tamar. Fares b...","Yawez fogel Seviz erw Deven, us Lenev. Seviz f..."
23216,MAT 1:4,MAT,1,4,Ram became the father of Amminadab. Amminadab ...,and Ram begat Amminadab; and Amminadab begat N...,And Aram begat Aminadab; and Aminadab begat Na...,And the son of Ram was Amminadab; and the son ...,Esrom bigat Aram. Aram bigat Amynadab. Amynada...,Izvun fogel Even. Even fogel Enjrewef. Enjrewe...
23217,MAT 1:5,MAT,1,5,Salmon became the father of Boaz by Rahab. Boa...,and Salmon begat Boaz of Rahab; and Boaz begat...,And Salmon begat Booz of Rachab; and Booz bega...,And the son of Salmon by Rahab was Boaz; and t...,"Salmon bigat Booz, of Raab. Booz bigat Obeth, ...","Zepnur fogel Fuud, us Veef. Fuud fogel Ufilx, ..."
...,...,...,...,...,...,...,...,...,...,...
26987,JHN 21:21,JHN,21,21,"Peter, seeing him, said to Jesus, “Lord, what ...","Peter therefore seeing him saith to Jesus, Lor...","Peter seeing him saith to Jesus, Lord, and wha...","Seeing him, Peter said to Jesus, What about th...","Therfor whanne Petre hadde seyn this, he seith...","Lxivsuv hxerri Qilvi xewwi zijr lxoz, xi ziolx..."
26988,JHN 21:22,JHN,21,22,"Jesus said to him, “If I desire that he stay u...","Jesus saith unto him, If I will that he tarry ...","Jesus saith unto him, If I will that he tarry ...","Jesus said to him, If it is my desire for him ...","Jhesus seith to him, So I wole that he dwelle ...","Yxizaz ziolx lu xon, Zu O hupi lxel xi whippi ..."
26989,JHN 21:23,JHN,21,23,This saying therefore went out among the broth...,This saying therefore went forth among the bre...,Then went this saying abroad among the brethre...,So this saying went about among the brothers t...,Therfor this word wente out among the brithere...,Lxivsuv lxoz huvw hirli ual enurg lxi fvolxivi...
26990,JHN 21:24,JHN,21,24,This is the disciple who testifies about these...,This is the disciple that beareth witness of t...,This is the disciple which testifieth of these...,This is the disciple who gives witness about t...,"This is thilke disciple, that berith witnessyn...","Lxoz oz lxopci wozkoqpi, lxel fivolx holrizzjr..."


In [40]:
df2 = pd.DataFrame({'prompt': train['engWycliffe'], 'completion': train['birrig']})
df2['completion'] = df2['completion'].apply(lambda x: " " + x + '\n###')
df2['prompt'] = df2['prompt'].apply(lambda x: x + "\n\n###\n\n")
df2.to_json('data/train.jsonl', orient='records', lines=True)


In [41]:

# set environment variable in ipython notebook
import os
os.environ["OPENAI_API_KEY"] = get_config('openai')['api_key']




In [43]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train.jsonl -q

Analyzing...

- Your file contains 3101 prompt-completion pairs
- There are 1 duplicated prompt-completion sets. These are rows: [403]
- All prompts end with suffix `\n\n###\n\n`
- All completions end with suffix `\n###`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 1 duplicate rows [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `data/train_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "data/train_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\n###\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.05 hours to train a `curie` model, and less for `ada` and `babbage`. Queu

In [44]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_prepared.jsonl"

Upload progress: 100%|███████████████████████| 845k/845k [00:00<00:00, 913Mit/s]
Uploaded file from data/train_prepared.jsonl: file-pizJI6isBlzPtwpqRT9dPzfH
Created fine-tune: ft-qhfzzyUAe2qEgFt92vYRcUpx
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-29 09:47:02] Created fine-tune: ft-qhfzzyUAe2qEgFt92vYRcUpx

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-qhfzzyUAe2qEgFt92vYRcUpx



In [45]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-qhfzzyUAe2qEgFt92vYRcUpx

[2023-03-29 09:47:02] Created fine-tune: ft-qhfzzyUAe2qEgFt92vYRcUpx
[2023-03-29 09:57:31] Fine-tune costs $3.91
[2023-03-29 09:57:31] Fine-tune enqueued. Queue number: 0
[2023-03-29 09:57:34] Fine-tune started
[2023-03-29 10:04:24] Completed epoch 1/4
[2023-03-29 10:10:07] Completed epoch 2/4
[2023-03-29 10:15:54] Completed epoch 3/4
[2023-03-29 10:21:39] Completed epoch 4/4
[2023-03-29 10:21:57] Uploaded model: curie:ft-personal-2023-03-29-17-21-57
[2023-03-29 10:21:58] Uploaded result file: file-cKNxGdllCJYvelM2RxwJeEEQ
[2023-03-29 10:21:58] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2023-03-29-17-21-57 -p <YOUR_PROMPT>


In [46]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

def translate(text):
    response = openai.Completion.create(
        model="curie:ft-personal-2023-03-29-17-21-57",
        prompt=text + "\n\n###\n\n",
        temperature=0.2,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n###"]
    )
    translation = response.get('choices',[{}])[0].get('text').strip()
    decoded_translation = substitution_cipher(translation, encode=False).strip()
    bleu_score = sentence_bleu([text.split()], decoded_translation.split())
    
    return (translation, decoded_translation, bleu_score)

In [48]:
translate("For God so loved me that he gave me new life in his kingdom")

/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


('Suv Guw zu puwpi ni lxel xi givi ni rih pofxor or xoz corgwun',
 'For God so lodle me that he gere me new libhin in his kingdom',
 6.965077358261036e-78)

In [50]:
translate("In the bigynnyng God made of nouyt heuene")

('Or lxi fogjrrjrg Guw newi us ruanl xiairi',
 'In the bigynnyng God made of noumt heuene',
 0.7071067811865475)

In [49]:
# Try some unknown words
translate("Bongo bongo I love you, gone to Venus with a hole in my shoe")

('Fungu fungu O puavi jua, guir lu Mivaz holx e xupi or nj zuri',
 'Bomgo bomgo I loure you, goen to Verus with a hole in my sone',
 0.29899503549981366)